### 2019年11月中华人民共和国县以上行政区划代码
- http://www.mca.gov.cn/article/sj/xzqh/2019/2019/201912251506.html
---
### 目标任务
- 产出结果1
    - 省编码/ 市编码/ 省名称/ 市名称
- 产出结果2
    - 省编码/ 市编码/ 区县编码/ 省名称/ 市名称/ 区县名称

In [ ]:
<tr height=19 style='mso-height-source:userset;height:14.25pt'>
<td height=19 class=xl6528029 style='height:14.25pt'></td>
<td class=xl7028029>110000</td>
<td class=xl7028029>北京市</td>
</tr>
<tr height=19 style='mso-height-source:userset;height:14.25pt'>
<td height=19 class=xl6528029 style='height:14.25pt'></td>
<td class=xl7128029>110101</td>
<td class=xl7128029><span style='mso-spacerun:yes'>   </span>东城区</td>
</tr>

## 产出结果1
### 解析 html && 结果合并

In [3]:
import requests
import warnings
import pandas as pd
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',5000)

xzqh_html = requests.get('http://www.mca.gov.cn/article/sj/xzqh/2019/2019/201912251506.html')
print("xzqh_html_status_code: " + str(xzqh_html.status_code))
soup = BeautifulSoup(xzqh_html.text, 'html.parser')

xzqh_codes = []
xzqh_names = []

for sp in soup.find_all(attrs={"class" : "xl7028029"}):
    ts = sp.getText().strip()
    if ts.isdigit():
        xzqh_codes.append(ts)
    else :
        xzqh_names.append(ts)
        
xzqh_code_names = [ [xzqh_code, xzqh_name] for xzqh_code, xzqh_name in zip(xzqh_codes, xzqh_names) ]

xzqh_html_status_code: 200


### 截至2017年12月31日，中国有34个省级行政区，包括23个省、5个自治区、4个直辖市、2个特别行政区。
- 23个省：河北省、山西省、辽宁省、吉林省、黑龙江省、江苏省、浙江省、安徽省、福建省、江西省、山东省、河南省、湖北省、湖南省、广东省、海南省、四川省、贵州省、云南省、陕西省、甘肃省、青海省、台湾省。
- 5个自治区：内蒙古自治区、广西壮族自治区、西藏自治区、宁夏回族自治区、新疆维吾尔自治区。
- 4个直辖市：北京市、天津市、上海市、重庆市。
- 2个特别行政区：香港特别行政区、澳门特别行政区。

In [4]:
# 转为 pandas DF 调整部分数据
xzqh_code_names_df = pd.DataFrame(xzqh_code_names, columns=['xzqh_code','xzqh_name'])
xzqh_code_names_df['xzqh_name'] = xzqh_code_names_df.xzqh_name.apply(lambda xzqh_name : '中国台湾省' if '台湾省' in xzqh_name else xzqh_name)
xzqh_code_names_df.head(3)

,xzqh_code,xzqh_name
0,110000,北京市
1,120000,天津市
2,130000,河北省


In [5]:
# 省份-直辖市-自治区-特别行政区处理
sf_zxs_zzq_tbxzqs = xzqh_code_names_df[xzqh_code_names_df.xzqh_code.str.endswith('0000')]
sf_zxs_zzq_tbxzqs['xzqh_ssq_code'] = sf_zxs_zzq_tbxzqs.xzqh_code.apply(lambda xzqh_code : xzqh_code[0:2])

xss = xzqh_code_names_df[(~xzqh_code_names_df.xzqh_code.str.endswith('0000')) & xzqh_code_names_df.xzqh_code.str.endswith('00')]
xss['xzqh_ssq_code'] = xss.xzqh_code.apply(lambda xzqh_code : xzqh_code[0:2])

# 省-下属市结果合并
union_df = pd.merge(sf_zxs_zzq_tbxzqs, xss, on='xzqh_ssq_code', how='left')
union_df.columns = ["province_code","province_name","xzqh_ssq_code","city_code","city_name"]

# 删除无效列
union_df = union_df.drop(columns='xzqh_ssq_code')


"""
空值列数据补充
"""
from numpy import NaN
def makeup_val(province_code,city_code):
    if city_code is NaN:
        return province_code
    else:
        return city_code
# 补充直辖市-特别行政区等空数据
union_df['city_code'] = union_df.apply(lambda rw: makeup_val(rw['province_code'],rw['city_code']),axis=1)
union_df['city_name'] = union_df.apply(lambda rw: makeup_val(rw['province_name'],rw['city_name']),axis=1)

# 格式化结果数据
union_df['province_code'] = union_df.province_code.apply(lambda province_code : province_code[0:2])
union_df['city_code'] = union_df.city_code.apply(lambda city_code : city_code[0:4])

# 导出结果性文件
union_df.to_csv("./2019年11月中华人民共和国县以上行政区划代码_省市_编码_中文名_340.csv",encoding='utf8',sep=',');

In [6]:
print(len(union_df.province_code.unique()), union_df.shape)

34 (340, 4)


In [10]:
union_df.head(3)

,province_code,province_name,city_code,city_name
0,11,北京市,1100,北京市
1,12,天津市,1200,天津市
2,13,河北省,1301,石家庄市


## 产出结果2
- 依赖结果1
- 省编码/ 市编码/ 区县编码/ 省名称/ 市名称/ 区县名称

In [15]:
total_xzqh_codes = []
total_xzqh_names = []

for sp in soup.find_all('td', attrs={"class" : ["xl7028029","xl7128029"]}):
    if 'xl7028029' in sp.decode():
        parent = sp.getText().strip()
        if parent.isdigit():
            parent_xzqh_codes = parent
        else :
            parent_xzqh_names = parent
    else :
        child = sp.getText().strip()
        if child.isdigit():
            total_xzqh_codes.append([parent_xzqh_codes,child])
        else :
            total_xzqh_names.append([parent_xzqh_names,child])
total_xzqh_code_names = [ xzqh_code+xzqh_name for xzqh_code,xzqh_name in zip(total_xzqh_codes, total_xzqh_names) ]

# 合并结果数据
total_xzqh_code_names_df = pd.DataFrame(total_xzqh_code_names,columns=['zxs_zzqd','ssqx','zxs_zzqd_name','ssqx_name'])
total_xzqh_code_names_df['city_code'] = total_xzqh_code_names_df.zxs_zzqd.apply(lambda zxs_zzqd : zxs_zzqd[:4])
total_xzqh = pd.merge(union_df,total_xzqh_code_names_df,on='city_code', how='left')

# 剔除无用列
total_xzqh = total_xzqh.drop(['zxs_zzqd','zxs_zzqd_name'],axis=1);

# 导出结果性文件
union_df.to_csv("./2019年11月中华人民共和国县以上行政区划代码_省市区县_2852.csv",encoding='utf8',sep=',');

In [17]:
total_xzqh.shape

(2852, 6)

In [16]:
total_xzqh.head(3)

,province_code,province_name,city_code,city_name,ssqx,ssqx_name
0,11,北京市,1100,北京市,110101,东城区
1,11,北京市,1100,北京市,110102,西城区
2,11,北京市,1100,北京市,110105,朝阳区
